In [1]:
import lionagi.libs.ln_func_call as func_call
from lionagi.libs.ln_async import AsyncUtil
from lionagi.core.messages.system import System
from lionagi.core.messages.instruction import Instruction

from lionagi.core.schema.structure import Structure

from lionagi import Branch
from lionagi.core.mail.schema import BaseMail
from lionagi.core.schema.base_node import BaseRelatableNode
from collections import deque
from typing import Any

from lionagi.core.agent.base_agent import BaseAgent
from lionagi.core.schema.action_node import ActionNode, ActionSelection
from lionagi.core.branch.executable_branch import ExecutableBranch

In [2]:
# class ExecutableBranch(BaseRelatableNode):

#     def __init__(self, **kwargs):
#         super().__init__()
#         self.branch: Branch = Branch(**kwargs)
#         self.pending_ins = {}           ###
#         self.pending_outs = deque()     ###
#         self.responses = []
#         self.execute_stop = False       ###
#         self.context = None             ###
    
#     def system_process(self, system: System):
#         self.branch.add_message(system=system)
#         print('system call:')

#     async def instruction_process(self, instruction: Instruction):
#         if self.context:
#             instruction.content.update({"context": self.context})
#             self.context=None
#         result = await self.branch.chat(instruction)
#         print('instruction call:', result)
#         self.responses.append(result)
    
#     async def action_process(self, action: ActionNode):
#         # instruction = action.instruction
#         # if self.context:
#         #     instruction.content.update({"context": self.context})
#         #     self.context=None
#         try:
#             func = getattr(self.branch, action.action)
#         except:
#             raise ValueError(f"{action.action} is not a valid action")

#         if action.tools:
#             self.branch.register_tools(action.tools)
#         # result = await func(instruction, tools=action.tools, **action.action_kwargs)
#         if self.context:
#             result = await func(action.instruction.content, context=self.context,
#                                 tools=action.tools, **action.action_kwargs)
#             self.context = None
#         else: 
#             result = await func(action.instruction.content, tools=action.tools, **action.action_kwargs)
#         print('action calls:', result)
#         self.responses.append(result)

#     async def agent_process(self, agent):
#         context = self.responses
#         result = await agent.execute(context)
#         self.context=result
#         print('agent call:', result)
#         self.responses.append(result)

#     def send(self, recipient_id: str, category: str, package: Any) -> None:
#         mail = BaseMail(sender_id=self.id_, 
#                         recipient_id=recipient_id, category=category, package=package)
#         self.pending_outs.append(mail)
    
#     async def process(self):
#         for key in list(self.pending_ins.keys()):
#             while self.pending_ins[key]:
#                 mail = self.pending_ins[key].popleft()
#                 if mail.category == "start":                                        ###
#                     start_mail_content = mail.package                               ###
#                     self.context = start_mail_content['context']                    ###
#                     self.send(start_mail_content['structure_id'], "start", "start") ###
#                 if mail.category == "node":
#                     if isinstance(mail.package, System):
#                         self.system_process(mail.package)
#                         self.send(mail.sender_id, "node_id", mail.package.id_)
#                     elif isinstance(mail.package, Instruction):
#                         await self.instruction_process(mail.package)
#                         self.send(mail.sender_id, "node_id", mail.package.id_)
#                     elif isinstance(mail.package, ActionNode):
#                         await self.action_process(mail.package)
#                         self.send(mail.sender_id, "node_id", mail.package.instruction.id_)
#                     elif isinstance(mail.package, BaseAgent):
#                         await self.agent_process(mail.package)
#                         self.send(mail.sender_id, "node_id", mail.package.id_)
#                 elif mail.category == "end":                                        ###                             
#                     self.execute_stop = True
#                     self.send(mail.sender_id, "end", "end")
    
#     async def execute(self, refresh_time=1):                                        ###
#         while not self.execute_stop:
#             await self.process()
#             await AsyncUtil.sleep(refresh_time)

In [3]:
from lionagi.core.tool.tool_manager import func_to_tool

In [4]:
def multiply(number1: float, number2: float):
    """
    Perform multiplication on two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.

    Returns:
        The product of number1 and number2.

    """
    return number1 * number2

tool_m = func_to_tool(multiply)

In [5]:
# action = ActionSelection(action='followup', action_kwargs={'auto': True})

In [6]:
sys_mul = System(system="you are asked to perform as a function picker and parameter provider", name='calculator')
instruction = Instruction(instruction="Think step by step, understand the following basic math question and provide parameters for function calling.")

In [7]:
question1 = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

import json
context = {"Question1": question1, "question2": question2}
context = json.dumps(context)

In [8]:
struct_mul = Structure()
struct_mul.add_node(sys_mul)
struct_mul.add_node(instruction)
struct_mul.add_node(tool_m[0])
# struct_mul.add_node(action)
struct_mul.add_relationship(sys_mul, instruction)
struct_mul.add_relationship(instruction, tool_m[0], bundle=True)   ### TODO:Enforce bundle for certain node types
# struct_mul.add_relationship(instruction, action, bundle=True)

In [9]:
## output_parser_func parameter: agent self

def multiply_output_parser(agent):
    return agent.executable.responses

In [10]:
calc = BaseAgent(structure=struct_mul, executable_class=ExecutableBranch, output_parser=multiply_output_parser, executable_class_kwargs={"name": "calculator"})

In [11]:
tool_m[0].id_

'09bf165b8d4277289592be5fc072bd75'

In [12]:
result = await calc.execute(context=context)

---------------Welcome: calculator------------------


system: you are asked to perform as a function picker and parameter provider

action calls: For the first question, we are looking for the number of unique kinds of balls and each kind comes in four different colors. The items listed are: basketball, football, backpack, water bottle, strawberry, tennis ball, rockets. Among these, the balls are: basketball, football, and tennis ball. So, we have 3 kinds of balls, each coming in 4 different colors.

For the second question, we have three fruits, each with 2 different colors, and we want to know how many unique kinds of fruits there are. Since each fruit is unique and the color variations do not change the kind of fruit, we simply have 3 fruits.

To provide parameters for function calling, we will use the multiply function to calculate the total number of unique items for each question based on their specific color variations.

1. For the first question, calculate the total number of unique balls considering their color variations.
2. For the second question, calculate the total number of unique fruits considering 

In [13]:
calc.executable.branch.messages

,node_id,timestamp,role,sender,content
0,8e84bffa1e0e8dded22e550c3dca79a9,2024_03_16T16_52_21_832805+00_00,system,system,"{""system_info"": ""you are asked to perform as a..."
1,bc26f770f61b5e77dcae1b7abe723129,2024_03_16T16_52_22_361829+00_00,user,user,"{""instruction"": {""instruction"": ""Think step by..."
2,8959172098c655abd1cf27fa22e314ad,2024_03_16T16_52_36_868727+00_00,assistant,calculator,"{""response"": ""For the first question, we are l..."


In [14]:
for i in calc.executable.branch.messages['content']:
    print(i)

{"system_info": "you are asked to perform as a function picker and parameter provider"}
{"instruction": {"instruction": "Think step by step, understand the following basic math question and provide parameters for function calling."}, "context": "{\"Question1\": \"There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?\", \"question2\": \"There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?\"}"}
{"response": "For the first question, we are looking for the number of unique kinds of balls and each kind comes in four different colors. The items listed are: basketball, football, backpack, water bottle, strawberry, tennis ball, rockets. Among these, the balls are: basketball, football, and tennis ball. So, we have 3 kinds of balls, each coming in 4 different colors.\n\nFor the second question, we have three fruits, each with